In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import nltk


In [39]:
# Genism word2vec Model Training
model = Word2Vec(food_vocab, min_count=1, size=300, workers=4, window=5, sg=0, hs=1)
word_vectors = model.wv
# summarize the loaded model
print(model)
vocab = list(model.wv.vocab)
#print(words)

Word2Vec(vocab=912, size=300, alpha=0.025)


In [27]:

from gensim.models import FastText

ftmodel = FastText(size=4, window=3, min_count=1, bucket=2)
ftmodel.build_vocab(sentences=food_vocab)
ftmodel.train(sentences=food_vocab, total_examples=len(food_vocab), epochs=10)
ftvocab = list(ftmodel.wv.vocab)


ftmodel.wv.similarity('blanc', 'blancs')


0.9999978656665574

In [ ]:
# FOOD PARSER

#matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
#patterns = list(nlp.tokenizer.pipe(foods.name))
#matcher.add("FOOD_PATTERN", None, *patterns)
tokenizer = Tokenizer(nlp.vocab)
products = list(nlp.pipe(data.description))
products = list(nlp.pipe(data.clean_description))

    
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = list(nlp.tokenizer.pipe(foods.name))
matcher.add("FOOD_PATTERN", None, *patterns)

food_parser = []

for line in products:
    matches = matcher(line)
    elements = []
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text)
        food_parser.append(max(elements, key=len).lower())
    else:
        text_blob_object = TextBlob(line.text)
        singular_line = ' '.join(text_blob_object.words.singularize())
        matches = matcher(tokenizer(singular_line))
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = tokenizer(singular_line)[start:end]
                elements.append(span.text)
            food_parser.append(max(elements, key=len).lower())
        else:
            plural_line = ' '.join(text_blob_object.words.pluralize())
            matches = matcher(tokenizer(plural_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(plural_line)[start:end]
                    elements.append(span.text)
                food_parser.append(max(elements, key=len).lower())
            else:
                ratios = process.extractOne(line.text.lower(), foods.name)
                if len(ratios) > 0 and ratios[1] > 85:
                    food_parser.append(ratios[0])
                else:
                    food_parser.append("")


data['food_match'] = food_parser
        

print(len(data[data.food_match == '']) / len(data))


In [ ]:
# FOOD PARSER IMPROVED WITH SHELVES / FOODCAT MAPPING

mapping_categories = pd.read_csv('/Users/vincentsalamand/Downloads/category_mapping.csv')

#matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
#patterns = list(nlp.tokenizer.pipe(foods.name))
#matcher.add("FOOD_PATTERN", None, *patterns)
tokenizer = Tokenizer(nlp.vocab)

for index, product_category in mapping_categories.iterrows():
    if product_category["shelter type"] == "shelter_parent":
        filter_data = data[data.shelter_parent == product_category.shelter].clean_description
        mapping_cat = mapping_categories[mapping_categories.shelter == product_category.shelter]["category ids"]
        
    elif product_category["shelter type"] == "shelter_child":
        filter_data = data[data.shelter_child == product_category.shelter].clean_description
        mapping_cat = mapping_categories[mapping_categories.shelter == product_category.shelter]["category ids"]   
        
    products = list(nlp.pipe(filter_data))
    
    mapping = mapping_cat.str.split(',', expand=True)
    
    if not mapping.isnull().values.any():
        cat_ids = [int(ids[1]) for ids in mapping.iteritems()]
        related_foods = foods[foods.category_id.isin(cat_ids)]
    else:
        related_foods = foods
    
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = list(nlp.tokenizer.pipe(related_foods.name))
    matcher.add("FOOD_PATTERN", None, *patterns)
    
    food_parser = []
    
    for line in products:
        matches = matcher(line)
        elements = []
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = line[start:end]
                elements.append(span.text)
            food_parser.append(max(elements, key=len).lower())
        else:
            text_blob_object = TextBlob(line.text)
            singular_line = ' '.join(text_blob_object.words.singularize())
            matches = matcher(tokenizer(singular_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(singular_line)[start:end]
                    elements.append(span.text)
                food_parser.append(max(elements, key=len).lower())
            else:
                plural_line = ' '.join(text_blob_object.words.pluralize())
                matches = matcher(tokenizer(plural_line))
                if len(matches) > 0:
                    for match_id, start, end in matches:
                        span = tokenizer(plural_line)[start:end]
                        elements.append(span.text)
                    food_parser.append(max(elements, key=len).lower())
                else:
                    ratios = process.extractOne(line.text.lower(), related_foods.name)
                    if len(ratios) > 0 and ratios[1] > 85:
                        food_parser.append(ratios[0])
                    else:
                        food_parser.append("")

    if product_category["shelter type"] == "shelter_parent":
        data.loc[data.shelter_parent == product_category.shelter, 'food_match'] = food_parser
    elif product_category["shelter type"] == "shelter_child":
        data.loc[data.shelter_child == product_category.shelter, 'food_match'] = food_parser
        

print(len(data[data.food_match == '']) / len(data))

In [ ]:
# check for foods unmatch in catalog
print(len(set(foods.name.str.lower().tolist()) - set(data.food_match.tolist())))
set(foods.name.str.lower().tolist()) - set(data.food_match.tolist())

In [ ]:
data[data.description.str.lower().str.contains("avocat")]


In [151]:
pd.read_csv('/Users/vincentsalamand/Downloads/leclerc_catalog.csv')

,quantity_match,unit_match,price_match,price_per_unit_match,is_promo,clean_description,brand,description,image_url,origin,...,promo_price,shelter_child,shelter_main,shelter_parent,url,shelter,ean,is_frozen,is_available,promo_price_per_unit_match
0,0.650,kg,8.84,13.60,False,plat grillé en et extra en herbes 5,NaN,Plateau grill Tendre et Plus Extra tendre - He...,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,8.84 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.265980e+12,False,True,8.84
1,0.320,kg,3.89,12.16,False,brochette porc en et extra en 3,NaN,Brochette porc Tendre Et Plus Extra tendre x4 ...,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,3.89 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.265980e+12,False,True,3.89
2,0.330,kg,2.97,9.00,False,saucisse aux herbes chipolata,NaN,Saucisses aux herbes Ferial Avec Chipolata,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,2.97 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.661112e+12,False,True,2.97
3,0.275,kg,4.07,14.80,False,saucisse chipolata 5 5,NaN,Saucisse Chipolatas Bio x5 - VPF - 275g,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,4.07 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.661113e+12,False,True,4.07
4,0.330,kg,3.55,10.76,False,chipolata la rouge 3,Bigard,Chipolatas label rouge Bigard x6 - 330g,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,3.55 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.273230e+12,False,True,3.55
5,0.330,kg,2.99,9.06,False,chipolata 5,Férial,Chipolatas Férial 6x - 55g,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,2.99 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.661112e+12,False,True,2.99
6,0.330,kg,3.10,9.39,False,chipolata 3,Socopa,Chipolatas superieure Socopa Sous atmosphère x...,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,3.10 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.039051e+12,False,True,3.10
7,0.330,kg,2.80,8.48,False,chipolata en et 3,NaN,Chipolatas Tendre Et Plus x6 - 330g,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,2.80 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.265981e+12,False,True,2.80
8,0.330,kg,3.69,11.18,False,chipolata 3,NaN,Chipolatas supérieures x6 - 330g,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,3.69 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.265980e+12,False,True,3.69
9,1.200,kg,8.03,6.69,False,chipolata au naturel,NaN,Chipolatas boyau naturel - VPF x24 -,https://fd4-photos.leclercdrive.fr/image.ashx?...,NaN,...,8.03 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...,"['Viandes Poissons', 'Boucherie', 'Saucisses e...",3.266111e+12,False,True,8.03
